# Declarative Widgets Walkthrough

## Introduction

Built on top of [IPyWidgets](https://github.com/ipython/ipywidgets) and combined with [Polymer](https://www.polymer-project.org/1.0/) and [Web Components](http://webcomponents.org/), these widgets use a declarative syntax for creating interactive areas that are usable throughout a notebook.

## Widgets

First we need to add, import, and initialize the widget system:

In [ ]:
// modify to IP and Port of this notebook server
%addjar http://localhost:8888/nbextensions/urth_widgets/urth-widgets.jar

In [ ]:
import urth.widgets._
initWidgets

Let's start by creating a "Hello world" function.

In [ ]:
def greet(name: String = "world") = s"Hello ${name}!"

Next, we'll bind to this function and modify the name field to update our greeting.

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function id="f" ref="greet" arg-name="{{name}}" result="{{greeting}}"></urth-core-function>
    <label>Name:</label> <input type="text" value="{{name::change}}"></input><br/>
</template> 

Note how the default value is set based on the argument passed into the function. We pass the name of our function to the `ref` parameter, then explicitly set the arguments with the `arg-` prefix, and finally bind our output (`result`).

In [ ]:
%%html
<template is="urth-core-bind">
    <button onClick="f.invoke()">invoke</button><br/>
    <span id="test1">{{greeting}}</span>
</template> 

Try changing the `Name` argument and clicking the button above to call the `greet` function. The resulting greeting updates based on the current `Name` field.

Click [here](../examples/urth-core-function.ipynb) to learn more about `<urth-core-function>`

We can also bind to variables over independent channels.

In [ ]:
%%html
<template is='urth-core-bind' channel='a'>
    <div>Hello <span id="test2">{{user}}</span></div>
    Name: <input value='{{user::input}}'></input>
</template>

In [ ]:
%%html
<template is='urth-core-bind' channel='b'>
    <div>Hello <span id="test3">{{user}}</span></div>
    Name: <input value='{{user::input}}'></input>
</template>

Try modifying the `user` defined in channel `a`. This change will not impact the user defined in channel `b`.

Now to something a bit more complex. What if you want to have Scala code that reacts to changes in a value on a template. Lets start with the template below:

In [ ]:
%%html
<template is='urth-core-bind' channel='c'>
    <div>Type something <input value='{{aSomething::input}}'></input></div>
</template>

Now we can create a Scala function that will watch for changes to the value.

In [ ]:
import urth.widgets.WidgetChannels.channel

val on_aSomething_change = (oldVal: Option[String], newVal: String) => {
    val msg = s"Hello from on_aNumber_change! Got ${newVal}"
    channel("c").set("message", msg)
}

In [ ]:
lazy val initChannelWatch = () => {
    channel("c").watch("aSomething", on_aSomething_change)
}

In [ ]:
%%html
<template is='urth-core-bind'>
    <urth-core-function id='icw' ref='initChannelWatch'></urth-core-function>
    <button onClick='icw.invoke()'>initChannelWatch</button><br/>
</template>

Lets create a template where we can set a message.

In [ ]:
%%html
<template is='urth-core-bind' channel='c'>
    <span id="test4">{{message}}</span>
</template>

Now when you type something on the input box, it triggers the Scala function `on_aSomething_change`. This function can then also set values on the channel by using the `set` method. Notice that the template with `{{message}}` is getting updated.

Click [here](../examples/urth-core-bind.ipynb) to learn more about `<urth-core-bind>`

Support for Spark DataFrames is provided. Below is a DataFrame with some basic contact information.

In [ ]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

In [ ]:
case class Contact(name: String, email: String)

In [ ]:
val df = sqlContext.createDataFrame(Seq(
    Contact("Jane Doe", "jane@doe.com"), 
    Contact("John Doe", "john@doe.com")))

Below, we print out the contents of the DataFrame in a more readable format.

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-dataframe id="f3" ref="df" value="{{contactInfo}}" auto></urth-core-dataframe>
    <template is="dom-repeat" items="{{contactInfo.data}}">
      <div class="info">
            <span class="test5">{{item.0}}</span>, <span>{{item.1}}</span>
      </div>
    </template>
</template>

By setting the `auto` keyword, the resulting output will update whenever the DataFrame is modified. Try changing which set of contact information is used.

In [ ]:
val df = sqlContext.createDataFrame(Seq(
        Contact("Richard Roe", "richard@roe.com"),
        Contact("Bob Murphy", "bob@murphy.com")))

Click [here](../examples/urth-core-dataframe.ipynb) to learn more about `<urth-core-dataframe>`

We can also import web components. Below, we bring in the `paper-input` element from the [Polymer Catalog](https://elements.polymer-project.org/).

In [ ]:
%%html
<link rel='import' href='urth_components/paper-input/paper-input.html' 
        is='urth-core-import' package='PolymerElements/paper-input'>
<paper-input label="Input label"></paper-input>

Click [here](../examples/urth-core-import.ipynb) to learn more about `<urth-core-import>`